### Copied Code From This location:
https://pypi.org/project/darkskylib/

### Dark sky attributes
https://darksky.net/dev/docs#api-request-types

### pip uninstall darksky
### pip install darkskylib


In [21]:
import pandas as pd

from darksky import forecast
from datetime import datetime as dt
from datetime import date

RAPIDAPI_KEY  = "KEY"

In [2]:
woodgulch = forecast(RAPIDAPI_KEY, 45.827, -120.229)

print(type(woodgulch))

WOODGULCH = RAPIDAPI_KEY, 45.827, -120.229


t = dt(2005, 7, 25, 6).isoformat() #year, month, day, hour

woodgulch = forecast(RAPIDAPI_KEY, 45.827, -120.229, time=t)
print(woodgulch.time)

print(woodgulch['currently'])
print(woodgulch.temperature)
print(woodgulch.humidity)
#print(woodgulch.pressure)
print(woodgulch.windSpeed)
             

<class 'darksky.forecast.Forecast'>
1122296400
{'time': 1122296400, 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 58.8, 'apparentTemperature': 58.8, 'dewPoint': 47.58, 'humidity': 0.66, 'windSpeed': 6.41, 'windGust': 11.14, 'windBearing': 274}
58.8
0.66
6.41


In [158]:
t = dt(2005, 7, 25, 16).isoformat() #year, month, day, hour

woodgulch = forecast(RAPIDAPI_KEY, 45.827, -120.229, time=t)
print(woodgulch.time)

print(woodgulch['currently'])

1122332400
{'time': 1122332400, 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 85.94, 'apparentTemperature': 85.94, 'dewPoint': 38.65, 'humidity': 0.19, 'windSpeed': 5.81, 'windGust': 6.99, 'windBearing': 91}


In [4]:
woodgulch.time

1122332400

In [5]:
woodgulch['currently'].keys()

dict_keys(['time', 'precipIntensity', 'precipProbability', 'temperature', 'apparentTemperature', 'dewPoint', 'humidity', 'windSpeed', 'windGust', 'windBearing'])

In [6]:
woodgulch_df = pd.DataFrame(woodgulch['currently'], index=[0])

In [159]:
woodgulch_df

,time,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windGust,windBearing
0,1122332400,0,0,85.94,85.94,38.65,0.19,5.81,6.99,91


## load the NASA WoodGulch into DF and append weather data

In [273]:
nasa2005_wg = pd.read_csv("woodgulch/NASA2005WoodGulch.csv")

In [274]:
nasa2005_wg.acq_date = pd.to_datetime(nasa2005_wg.acq_date)

In [275]:
nasa2005_wg['year'] = pd.DatetimeIndex(nasa2005_wg.acq_date).year
nasa2005_wg['month'] = pd.DatetimeIndex(nasa2005_wg.acq_date).month
nasa2005_wg['day'] = pd.DatetimeIndex(nasa2005_wg.acq_date).day
nasa2005_wg['hour'] = (nasa2005_wg.acq_time/100).round().astype(int)
nasa2005_wg['datetime'] = pd.to_datetime(nasa2005_wg[['year','month','day','hour']])

In [276]:
nasa2005_wg.head()

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,version,bright_t31,frp,daynight,type,year,month,day,hour,datetime
0,343878,45.8023,-120.2710,336.1,1.3,1.1,2005-07-23,2040,Aqua,MODIS,...,6.2,318.8,17.3,D,0,2005,7,23,20,2005-07-23 20:00:00
1,344092,45.8270,-120.2290,314.7,1.1,1.1,2005-07-24,608,Terra,MODIS,...,6.2,292.8,15.2,N,0,2005,7,24,6,2005-07-24 06:00:00
2,344093,45.8176,-120.2265,307.7,1.1,1.1,2005-07-24,608,Terra,MODIS,...,6.2,293.2,9.1,N,0,2005,7,24,6,2005-07-24 06:00:00
3,344094,45.8082,-120.2242,318.2,1.1,1.1,2005-07-24,608,Terra,MODIS,...,6.2,293.9,18.6,N,0,2005,7,24,6,2005-07-24 06:00:00
4,344194,45.8062,-120.2258,300.2,1.1,1.0,2005-07-24,1015,Aqua,MODIS,...,6.2,287.7,5.6,N,0,2005,7,24,10,2005-07-24 10:00:00


In [277]:
def weather_lookup(df):
    data = []
    for index, row in df.iterrows():
        ts = dt(row['year'], row['month'], row['day'], row['hour']).isoformat() #year, month, day, hour
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(RAPIDAPI_KEY, lat, lon, time=ts)
        w_dict = {
            'Unnamed: 0': row['Unnamed: 0'],
            'timestamp': ts,
            'latitude': lat,
            'longitude': lon,
            'time': weather.time,
            'precipIntensity': weather.precipIntensity,
            'precipProbability': weather.precipProbability,
            'temperature': weather.temperature,
            'apparentTemperature': weather.apparentTemperature,
            'dewPoint': weather.dewPoint,
            'humidity': weather.humidity,
            'windSpeed': weather.windSpeed,
            'windGust': weather.windGust,
            'windBearing': weather.windBearing
        }
        data.append(w_dict)
    return data


In [278]:
weather_df = pd.DataFrame(weather_lookup(nasa2005_wg))

In [279]:
weather_df

,Unnamed: 0,timestamp,latitude,longitude,time,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windGust,windBearing
0,343878,2005-07-23T20:00:00,45.8023,-120.2710,1122174000,0,0,81.67,81.67,42.52,0.25,12.55,18.68,251
1,344092,2005-07-24T06:00:00,45.8270,-120.2290,1122210000,0,0,59.40,59.40,46.13,0.61,6.39,9.84,228
2,344093,2005-07-24T06:00:00,45.8176,-120.2265,1122210000,0,0,59.98,59.98,46.20,0.60,6.28,9.74,227
3,344094,2005-07-24T06:00:00,45.8082,-120.2242,1122210000,0,0,60.52,60.52,46.26,0.59,6.18,9.64,227
4,344194,2005-07-24T10:00:00,45.8062,-120.2258,1122224400,0,0,69.60,69.60,44.74,0.41,9.13,13.73,244
5,344195,2005-07-24T10:00:00,45.8040,-120.2123,1122224400,0,0,69.37,69.37,44.69,0.41,9.16,13.80,244
6,344196,2005-07-24T10:00:00,45.7949,-120.2153,1122224400,0,0,69.90,69.90,44.66,0.40,9.41,14.09,243
7,344418,2005-07-24T21:00:00,45.8336,-120.2225,1122264000,0,0,71.27,71.27,50.14,0.47,15.94,19.75,255
8,344419,2005-07-24T21:00:00,45.8301,-120.2617,1122264000,0,0,71.49,71.49,49.99,0.47,15.99,19.86,254
9,344420,2005-07-24T21:00:00,45.8319,-120.2418,1122264000,0,0,71.87,71.87,50.08,0.46,15.99,19.83,255


In [280]:
nasa2005_wg = nasa2005_wg.join(weather_df, lsuffix='_f', rsuffix='_w')

In [281]:
nasa2005_wg

,Unnamed: 0_f,latitude_f,longitude_f,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,time,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,windSpeed,windGust,windBearing
0,343878,45.8023,-120.2710,336.1,1.3,1.1,2005-07-23,2040,Aqua,MODIS,...,1122174000,0,0,81.67,81.67,42.52,0.25,12.55,18.68,251
1,344092,45.8270,-120.2290,314.7,1.1,1.1,2005-07-24,608,Terra,MODIS,...,1122210000,0,0,59.40,59.40,46.13,0.61,6.39,9.84,228
2,344093,45.8176,-120.2265,307.7,1.1,1.1,2005-07-24,608,Terra,MODIS,...,1122210000,0,0,59.98,59.98,46.20,0.60,6.28,9.74,227
3,344094,45.8082,-120.2242,318.2,1.1,1.1,2005-07-24,608,Terra,MODIS,...,1122210000,0,0,60.52,60.52,46.26,0.59,6.18,9.64,227
4,344194,45.8062,-120.2258,300.2,1.1,1.0,2005-07-24,1015,Aqua,MODIS,...,1122224400,0,0,69.60,69.60,44.74,0.41,9.13,13.73,244
5,344195,45.8040,-120.2123,308.8,1.1,1.0,2005-07-24,1015,Aqua,MODIS,...,1122224400,0,0,69.37,69.37,44.69,0.41,9.16,13.80,244
6,344196,45.7949,-120.2153,318.0,1.1,1.0,2005-07-24,1015,Aqua,MODIS,...,1122224400,0,0,69.90,69.90,44.66,0.40,9.41,14.09,243
7,344418,45.8336,-120.2225,335.3,1.5,1.2,2005-07-24,2123,Aqua,MODIS,...,1122264000,0,0,71.27,71.27,50.14,0.47,15.94,19.75,255
8,344419,45.8301,-120.2617,340.8,1.5,1.2,2005-07-24,2123,Aqua,MODIS,...,1122264000,0,0,71.49,71.49,49.99,0.47,15.99,19.86,254
9,344420,45.8319,-120.2418,383.9,1.5,1.2,2005-07-24,2123,Aqua,MODIS,...,1122264000,0,0,71.87,71.87,50.08,0.46,15.99,19.83,255
